# 1. import module into python

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets
import os
import sys
import warnings
import pickle
import datetime
from tabletree import TableNode, TableTree, TableLink

# 2. load data into python

In [2]:
with open('sah_dataset/ehr_tree.pkl', 'rb') as f:
    ehr_tree = pickle.load(f)

In [5]:
ehr_tree

In [ ]:
# ehr_tree have three tables: patient,encounter and diagnosis 
ehr_tree.getTN('patient').df.head()

In [ ]:
ehr_tree.getTN('encounter').df.head()

In [ ]:
enc_df['discharge_disposition_id'].value_counts()

In [ ]:
ehr_tree.getTN('diagnosis').df.head()

In [3]:
pat_df = ehr_tree.getTN('patient').df
enc_df = ehr_tree.getTN('encounter').df
dia_df = ehr_tree.getTN('diagnosis').df

In [ ]:
enc_df.loc[enc_df['encounter_id'] == 154352309]

In [ ]:
dia_df.loc[dia_df['encounter_id'] == 154352309]

In [ ]:
pat_df.head(10)
pat_df.dtypes
pat_df.sort_values(by='patient_sk')
## find that one patient_sk might have different patient_id

# 3.merge three tables

In [ ]:
#first merge diagnosis table to encounter table(remove some columns we don't need)
enc_dia_df = pd.merge(enc_df[['encounter_id','patient_id','admitted_dt_tm','discharged_dt_tm']],
                      dia_df[['encounter_id','diagnosis_id','diagnosis_type_id']],
                      on = 'encounter_id',
                      how = "left",
                      indicator=True
                     )
enc_dia_df.head()

In [ ]:
enc_dia_df2['_merge'].value_counts()

In [ ]:
enc_dia_df.shape

In [ ]:
enc_dia_df2.shape

In [ ]:
enc_dia_df.head()

In [ ]:
## check merge results, notice 117329 NA for diagnosis_id and diagnosis_type_id(means patient without diagnosis)
## for admitted_df 22543 NA means 22543 encounter have no record of time enter the study
enc_dia_df.isnull().sum()

In [ ]:
##also from "_merge" column can see 117329 row not match to any diagnosis id
enc_dia_df['_merge'].value_counts()

In [ ]:
enc_dia_df.drop("_merge", axis=1,inplace=True)

In [ ]:
enc_dia_df.head()

In [ ]:
## then merge enc_dia_df with patients_df table
pat_enc_dia_df = pd.merge(pat_df[['patient_id','patient_sk']],
                          enc_dia_df,
                          on = 'patient_id',
                          how = "left",
                          indicator=True
                         )
pat_enc_dia_df.head()

In [ ]:
pat_enc_dia_df.head(30)

In [ ]:
c = pat_enc_dia_df['diagnosis_id'].unique()

In [ ]:
len(c)

In [ ]:
## no missing data this time, which means every patient has a matched encounter id
pat_enc_dia_df.isnull().sum()

In [ ]:
##also from check '_merge' column show same result
pat_enc_dia_df['_merge'].value_counts()

In [ ]:
## now remove rows with NA
pat_enc_dia_df.dropna(inplace=True)

In [ ]:
pat_enc_dia_df.drop("_merge", axis=1,inplace=True)

In [ ]:
##split dataframe into many dataframes, each one is for one patient_sk, storage dfs into a list
pat_enc_dia_ls = [df for each_patient, df in pat_enc_dia_df.groupby("patient_sk")]

In [ ]:
pat_enc_dia_ls[0].head()

# 4. sort descending, convert time format, use latest time as 0

In [ ]:
#for each df, set the lasted discharged time to be 0
for i in range (0, len(pat_enc_dia_ls)):
    a = pat_enc_dia_ls[i]
    #sort pat_enc_dia_ls[i] by time
    a.sort_values('discharged_dt_tm', ascending = False, inplace = True)
    a = a.reset_index(drop=True)
    #### convert time column into datetime format
    a['discharged_dt_tm'] = pd.to_datetime(a['discharged_dt_tm'], format='%Y-%m-%d %H:%M:%S')
    a['time_diff'] = a.loc[0,'discharged_dt_tm'] - a['discharged_dt_tm']
    a['time_diff'] = a['time_diff']/np.timedelta64(1,'D')
    a['time_diff'] = round(a['time_diff'])
    pat_enc_dia_ls[i] = a

In [ ]:
pat_enc_dia_ls[0]

In [ ]:
pat_enc_dia_ls[90]

# 5. count top 200 diagnosis id

In [ ]:
##subset to get dia_df with 200 diag id

In [28]:
dia_id_dict = dia_df['diagnosis_id'].value_counts()

In [29]:
dia_id = dia_id_dict.keys()

In [30]:
top_dia_id = dia_id[0:200]

In [31]:
top_dia_df = dia_df.loc[dia_df['diagnosis_id'].isin(top_dia_id)]

In [ ]:
top_dia_df.head()

In [ ]:
pat_df.head()

In [ ]:
enc_df.head()

In [4]:
#merge encounter and patient table for medgan
pat_enc_df = pd.merge(pat_df,
                      enc_df[['encounter_id','patient_id','discharge_disposition_id','discharged_dt_tm']],
                      on = 'patient_id',
                      how = "left"
                     )

In [ ]:
pat_enc_df.head()

# add column for deadOrNot

In [5]:
pat_enc_df["deadOrAlive"]=pat_enc_df["discharge_disposition_id"].isin([19.0,20.0,11.0,21.0]) 

In [33]:
pat_enc_df.head()

,patient_id,patient_sk,race,gender,marital_status,encounter_id,discharge_disposition_id,discharged_dt_tm,deadOrAlive
0,291153,109343809,African American,Male,Unknown,528937.0,1.0,2011-07-21 16:50:00,False
1,291153,109343809,African American,Male,Unknown,154352309.0,1.0,2011-06-10 23:59:00,False
2,154555,109408557,Caucasian,Female,Single,479956.0,1.0,2010-11-03 23:59:00,False
3,154555,109408557,Caucasian,Female,Single,154355225.0,1.0,2011-11-12 23:59:00,False
4,154555,109408557,Caucasian,Female,Single,154893936.0,1.0,2010-12-13 23:59:00,False


In [43]:
pat_enc_df['race'].value_counts()

Caucasian                 283756
African American          100575
Other                      14759
Native American             7982
Asian                       7910
Unknown                     6285
Hispanic                    3520
Not Mapped                  2105
Biracial                    1637
Pacific Islander             656
Mid Eastern Indian           220
Null                         116
Asian/Pacific Islander        95
Name: race, dtype: int64

In [44]:
pat_enc_df['marital_status'].value_counts()

Married              163283
Single               132204
Widowed               62240
Divorced              39941
Null                  11402
Legally Separated      6582
Unknown                6178
NOT MAPPED             4848
Life Partner           1633
Not Mapped                1
Name: marital_status, dtype: int64

In [24]:
pat_enc_df.to_csv("gan_pat_df.csv")

In [32]:
top_dia_df.to_csv("top200diag.csv")